In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import fnmatch
import string
from urllib.parse import urlparse
import contractions

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

from deep_translator import GoogleTranslator

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
tweets_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Twitter_Data_Analysis/v2/data/tweets_v2.csv')
print(tweets_df.shape)
tweets_df.head()

(26015, 4)


,Unnamed: 0,date_created,tweet,city
0,0,2022-12-12 16:13:45+00:00,@esichq @byadavbjp @Rameswar_Teli @mygovindia ...,Mumbai
1,1,2022-12-10 06:30:56+00:00,@PotholeWarriors @CMOMaharashtra @mieknathshin...,Mumbai
2,2,2022-11-23 13:09:18+00:00,@Iam_Ayushmann Govandi is one of the Hotspot o...,Mumbai
3,3,2022-10-27 15:58:11+00:00,Till when medical negligence will exist in gov...,Mumbai
4,4,2022-07-28 03:03:15+00:00,Me being a doctor reading this\nAlso governmen...,Mumbai


In [3]:
tweets_df.rename(columns = {'Unnamed: 0':'tweet_id'}, inplace=True)
print(tweets_df.dtypes)

tweet_id         int64
date_created    object
tweet           object
city            object
dtype: object


In [4]:
#selecting only year from date_created column
tweets_df['date_created'] = pd.to_datetime(tweets_df['date_created'])
tweets_df['year'] = tweets_df['date_created'].dt.year
tweets_df.drop(['date_created'], axis=1, inplace=True)
tweets_df.head()

,tweet_id,tweet,city,year
0,0,@esichq @byadavbjp @Rameswar_Teli @mygovindia ...,Mumbai,2022
1,1,@PotholeWarriors @CMOMaharashtra @mieknathshin...,Mumbai,2022
2,2,@Iam_Ayushmann Govandi is one of the Hotspot o...,Mumbai,2022
3,3,Till when medical negligence will exist in gov...,Mumbai,2022
4,4,Me being a doctor reading this\nAlso governmen...,Mumbai,2022


In [5]:
print(tweets_df.city.value_counts())
print(tweets_df.year.value_counts())

city
Delhi        10005
Mumbai        6715
Hyderabad     3597
Bangalore     3229
Kolkata       1413
Chennai       1056
Name: count, dtype: int64
year
2020    8963
2021    7255
2019    3885
2022    3214
2018    2698
Name: count, dtype: int64


In [6]:
print('Shape of dataset before removal of duplicates is {}'.format(tweets_df.shape))
tweets_no_dupl_df = tweets_df.drop_duplicates(subset=['tweet'])
print('Shape of dataset after removal of duplicates is {}'.format(tweets_no_dupl_df.shape))

Shape of dataset before removal of duplicates is (26015, 4)
Shape of dataset after removal of duplicates is (19350, 4)


In [7]:
print(tweets_no_dupl_df.city.value_counts())
print(tweets_no_dupl_df.year.value_counts())

city
Delhi        7379
Mumbai       4963
Hyderabad    2614
Bangalore    2471
Kolkata      1077
Chennai       846
Name: count, dtype: int64
year
2020    6517
2021    5515
2019    2767
2022    2454
2018    2097
Name: count, dtype: int64


In [11]:
abbs_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Twitter_Data_Analysis/v2/data/text_abbreviations_v2.csv')
# abbs_df.drop('Unnamed: 0', axis=1, inplace=True)
abbs_df.text = abbs_df.text.str.lower()

abbs_dict = zip(abbs_df.text, abbs_df.meaning)
abbs_dict = list(abbs_dict)
abbs_dict = dict(abbs_dict)
abbs_dict

{';s': 'Gentle warning, like "Hmm? What did you say?"',
 '?': "I don't understand what you mean",
 '?4u': 'I have a question for you',
 '@teotd': 'At the end of the day',
 '*s*': 'Meaning "smile"',
 '*w*': 'Meaning "wink"',
 '^^': 'Meaning "read line" or "read message" above',
 '^5': 'High-five',
 '^urs': 'Up yours',
 '<3': 'Meaning "sideways heart" (love, friendship)',
 '<3333': 'Meaning "heart or love" (more 3s is a bigger heart)',
 '10m': 'Ten man as in ten man squad in online gaming',
 '10q': 'Thank you',
 '10x': 'Thanks',
 '14aa41': 'One for all, and all for one',
 '1ce': 'Once',
 '1dr': 'I wonder',
 '1nam': 'One in a million',
 '1tg, 2tg': '1 to go or 2 to go. Meaning number of items needed for win (online bingo)',
 '1up': 'Meaning extra life (online gaming)',
 '2b': 'To be',
 '2ez': 'Too easy',
 '2g2bt': 'Too good to be true',
 '2m2h': 'Too much too handle',
 '2mi': 'Too much information',
 '2mor': 'Tomorrow',
 '2moro': 'Tomorrow',
 '2n8': 'Tonight',
 '2nte': 'Tonight',
 '4ao': 

In [12]:

def removing_links(df):
     for tweets in df.loc[:,'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
        
          tokens = tokenizer.tokenize(tweets)
          x = [word for word in tokens if not urlparse(word).scheme]
          tweets = ' '.join(x)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def replacing_abbr(df, dictry):
    for tweets in df.loc[:,'tweet']:
          
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          tweets = tweets.split()
          tweets = [dictry[word] if word in dictry else word for word in tweets]
          tweets = ' '.join(tweets)
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
    return df

def contractions_handling(df):
     for tweets in df.loc[:,'tweet']:
          
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          tweets = contractions.fix(tweets)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def adding_space_bw_words_punc(df):
     for tweets in df.loc[:, 'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          tweets = tweets.replace(',', ' , ').replace('.', ' . ').replace('?', ' ? ').replace('!', ' ! ').replace('-', ' - ').replace('(', ' ( ').replace(')', ' ) ').replace(':', ' : ').replace(';', ' ; ').replace('"', ' " ').replace("'", " ' ").replace('  ', ' ')
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
     return df

def removing_hashtags_mentions(df):
     for tweets in df.loc[:, 'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tokens = tokenizer.tokenize(tweets)
          tokens = [word for word in tokens if word[0] not in ('#', '@')]
          tokens = [word for word in tokens if word[0] not in ('▪')]
          tweets = ' '.join(tokens)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
     return df
          
def removing_punctuations_emojis(df):
     for tweets in df.loc[:, 'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tweets = tweets.translate(str.maketrans(' ', ' ', string.punctuation)) # removes punctuations
          tweets = re.sub(r'[^\x00-\x7F]+', ' ', tweets) # removes emojis
          tweets = tweets.lower() # converts text to lower case
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
     return df

def removing_numbers(df):
     for tweets in df.loc[:, 'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          #remove numbers
          tweets = re.sub(r'\d+', '', tweets)
          tweets = re.sub(' +', ' ', tweets)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
     return df

def removing_stopwords(df):
     for tweets in df.loc[:,'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tokens = tokenizer.tokenize(tweets)
          filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]
          tweets = ' '.join(filtered_tokens)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def lemmatization(df):
     for tweets in df.loc[:,'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tokens = tokenizer.tokenize(tweets)
          lemmatizer = WordNetLemmatizer()
          lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
          tweets = ' '.join(lemmatized_tokens)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def removing_characters_less_3(df):
     for tweets in df.loc[:,'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tokens = tokenizer.tokenize(tweets)
          filtered_tokens = [word for word in tokens if len(word) > 2]
          tweets = ' '.join(filtered_tokens)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def removing_tweets_less_5(df):
     for tweets in df.loc[:, 'tweet']:
          
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          if len(tweets.split()) < 5:
               df.drop(df[df['tweet_id']==tweet_id].index, inplace=True)
     return df

def removing_duplicates(df):
     # removing duplicate tweets
     df = df.drop_duplicates(subset=['tweet'])
     
     # removing tweets that have the same first 5 words as the adjacent tweet
     for i in range(len(df)-1):
          try:
               if df['tweet'].iloc[i].split()[:5] == df['tweet'].iloc[i+1].split()[:5]:
                    df.drop(df[df['tweet_id']==df['tweet_id'].iloc[i+1]].index, inplace=True)
                    i -= 1
               else:
                    i += 1
          except:
               continue
               
     return df



def data_preprocessing(df, dictry):
     
     df = removing_links(df)
     df = replacing_abbr(df, dictry)
     df = contractions_handling(df)
     df = adding_space_bw_words_punc(df)
     df = removing_hashtags_mentions(df)
     df = removing_punctuations_emojis(df)
     # df = removing_numbers(df)
     df = lemmatization(df)
     # df = removing_stopwords(df)
     # df = removing_characters_less_3(df)
     df = removing_tweets_less_5(df)
     df = removing_duplicates(df)
     df.reset_index(drop=True, inplace=True)
     
     return df

In [13]:
tweets_cleaned_df = data_preprocessing(tweets_no_dupl_df, abbs_dict)
print(tweets_cleaned_df.shape)
tweets_cleaned_df.head()

/var/folders/mc/_plzx65559n4fxtnmj3tqk9w0000gp/T/ipykernel_28173/2739091423.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['tweet_id']==tweet_id].index, inplace=True)
/var/folders/mc/_plzx65559n4fxtnmj3tqk9w0000gp/T/ipykernel_28173/2739091423.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['tweet_id']==df['tweet_id'].iloc[i+1]].index, inplace=True)


(18146, 4)


,tweet_id,tweet,city,year
0,0,it is very bad thing to say that government di...,Mumbai,2022
1,1,all netas and their family should be admitted ...,Mumbai,2022
2,2,govandi is one of the hotspot of a well other ...,Mumbai,2022
3,3,till when medical negligence will exist in gov...,Mumbai,2022
4,4,me being a doctor reading this also government...,Mumbai,2022


In [14]:
# creating csv of cleaned dataset
pd.DataFrame.to_csv(tweets_cleaned_df, '/Users/nitanshjain/Documents/Projects/Twitter_Data_Analysis/v2/data/tweets_cleaned_v2.csv', index=False)

# Handing Hinglish


In [17]:
def hinglish_to_english(df, lang):
    translated_val = list()
    count = 0
    # translating the text to english using GoogleTranslator API
    for x in df['tweet']:
        count += 1
        try:
            # GoogleTranslator API has a limit of 5000 characters per request, so splitting the text into chunks of 5000 characters and then translating it
            if len(str(x))<5000:
                
                translation = GoogleTranslator(source=lang, target='en').translate(x)
                translated_val.append(translation)
            
            elif len(str(x))>5000 and len(str(x))<10000:
                
                split_x = [x[i:i+4999] for i in range(0, len(x), 4999)]
                translation_1 = GoogleTranslator(source=lang, target='en').translate(split_x[0])
                translation_2 = GoogleTranslator(source=lang, target='en').translate(split_x[1])
                translation = translation_1 + translation_2
                translated_val.append(translation)
                
        except Exception as e:
            # if the text is not in the language provided, then it will return a nan value or text length is more than 15000 characters
            translated_val.append(np.nan)
        if count%1000==0:
            print(count)
    
    # replacing the original text with the translated text
    df['tweet_translated'] = translated_val
    
    # returning the updated dataframe
    return df
            

In [18]:
translated_df = hinglish_to_english(tweets_cleaned_df, 'hi')
print(translated_df.isna().sum())
translated_df.dropna(inplace=True)
print(translated_df.isna().sum())
translated_df.head()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


/var/folders/mc/_plzx65559n4fxtnmj3tqk9w0000gp/T/ipykernel_30476/1205797308.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_translated'] = translated_val


,tweet_id,tweet,city,year,tweet_translated
0,0,bad thing say government medical spare operati...,Mumbai,2022,bad thing say government medical spare operati...
1,1,netas family admitted government hospital priv...,Mumbai,2022,netas family admitted government hospital priv...
2,2,govandi one hotspot well respiratory disease d...,Mumbai,2022,govandi one hotspot well respiratory disease f...
3,3,till medical negligence exist government hospi...,Mumbai,2022,till medical negligence exist government hospi...
4,4,doctor reading also government hospital resident,Mumbai,2022,doctor reading also government hospital resident


In [19]:
pd.DataFrame.to_csv(translated_df, '/Users/nitanshjain/Documents/Projects/Twitter_Data_Analysis/v2/data/tweets_cleaned_translated_v2.csv', index=False)